<b><font color="SteelBlue" size="+3">Anticipez les besoins en consommation de bâtiments 2</font></b>

Ce notebook est la suite du notebook d'exploration des données Deveau_Estelle_1_notebook_exploratoire_022024

# Introduction

## Imports

In [1]:
# Chargement des librairies
# Builtin
import os

# Data
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
import time
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate

# hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

## Data

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'Council District Map - 2024 - FULL.png',
 'data',
 'Deveau_Estelle_1_notebook_exploratoire_022024-svg.ipynb',
 'Deveau_Estelle_1_notebook_exploratoire_022024.ipynb',
 'Deveau_Estelle_2_notebook_prediction_022024.ipynb',
 'Map_of_Seattle,_divided_by_districts.png',
 'plan-codes-postaux -seattle.jpg']

In [3]:
os.listdir("data/cleaned/")

['df_cleaned.csv']

In [4]:
path     = "./data/cleaned/"
filename = "df_cleaned.csv"

In [5]:
df = pd.read_csv(path + filename)
df.head()

,Log_TotalGHGEmissions,Log_SiteEnergyUseWN,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseTypeGFA_pct,SecondLargestPropertyUseTypeGFA_pct,ThirdLargestPropertyUseTypeGFA_pct,BuildingAge,GroupedNeighborhood,GroupedLargType,GroupedSecondLargType,GroupedThirdLargType,SteamUse_pct,Electricity_pct,NaturalGas_pct,ENERGYSTARScore
0,5.521381,15.824652,1.0,12,88434,100.000000,0.000000,0.0,89,A,C,D,D,26.872820,52.917723,17.117720,60.0
1,5.689886,15.974742,1.0,11,103566,80.991831,4.462855,0.0,20,A,C,B,D,0.000000,37.426959,59.381320,61.0
2,5.657494,15.753792,1.0,10,61320,100.000000,0.000000,0.0,90,A,C,D,D,31.877211,39.858983,26.072621,56.0
3,5.171279,15.617677,1.0,11,83008,100.000000,0.000000,0.0,90,A,C,D,D,0.000000,46.368511,48.617731,27.0
4,5.400468,15.771071,1.0,8,102761,100.000000,0.000000,0.0,90,A,C,D,D,32.206065,51.453254,5.449851,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Log_TotalGHGEmissions                978 non-null    float64
 1   Log_SiteEnergyUseWN                  978 non-null    float64
 2   NumberofBuildings                    978 non-null    float64
 3   NumberofFloors                       978 non-null    int64  
 4   PropertyGFATotal                     978 non-null    int64  
 5   LargestPropertyUseTypeGFA_pct        978 non-null    float64
 6   SecondLargestPropertyUseTypeGFA_pct  978 non-null    float64
 7   ThirdLargestPropertyUseTypeGFA_pct   978 non-null    float64
 8   BuildingAge                          978 non-null    int64  
 9   GroupedNeighborhood                  978 non-null    object 
 10  GroupedLargType                      978 non-null    object 
 11  GroupedSecondLargType           

In [7]:
df.describe()

,Log_TotalGHGEmissions,Log_SiteEnergyUseWN,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseTypeGFA_pct,SecondLargestPropertyUseTypeGFA_pct,ThirdLargestPropertyUseTypeGFA_pct,BuildingAge,SteamUse_pct,Electricity_pct,NaturalGas_pct,ENERGYSTARScore
count,978.000000,978.000000,978.000000,978.000000,9.780000e+02,978.000000,978.000000,978.000000,978.000000,978.000000,978.000000,978.000000,607.000000
mean,3.781823,14.734266,1.165644,3.289366,8.004761e+04,89.575144,8.275799,1.592151,59.470348,1.822093,64.056055,28.618218,62.492586
std,1.479332,1.221480,1.398627,5.384415,1.399829e+05,17.067776,13.959109,5.136950,30.825019,8.232404,27.604363,24.587454,28.950331
min,-0.916291,10.970165,1.000000,1.000000,1.128500e+04,33.333333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,2.864054,13.920258,1.000000,1.000000,2.615650e+04,83.978087,0.000000,0.000000,37.000000,0.000000,41.442182,0.000000,43.000000
50%,3.773678,14.584837,1.000000,2.000000,3.997750e+04,100.000000,0.000000,0.000000,56.000000,0.000000,61.968374,28.608761,69.000000
75%,4.782730,15.495456,1.000000,3.000000,7.315975e+04,100.000000,12.910971,0.000000,88.000000,0.000000,93.615020,48.939649,87.500000
max,9.005223,19.431285,27.000000,76.000000,1.952220e+06,100.000000,50.000000,33.333333,116.000000,63.453768,103.872233,100.000000,100.000000


Nous allons commencer par chercher un modèle de ML pour prédire le SiteEnergyUseWN

Dans un premier temps, nous allons travailler en excuant l'ENERGYSTARScore puis nous ferons une comparaison avec son utilisation.

# Modelisation sans l'Energy Star Score

## Data preparation

### Séparation des données

In [8]:
# Sélection des variables explicatives et des variables cibles
features = df.drop(['Log_TotalGHGEmissions', 'Log_SiteEnergyUseWN', 'ENERGYSTARScore'], axis=1)
target = df['Log_SiteEnergyUseWN']

# Transformation des variables catégorielles en variables numériques (encodage one-hot)
features = pd.get_dummies(features)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
# Conservez les noms des colonnes dans une variable avant la mise à l'échelle
column_names = X_train.columns

# Affichage des dimensions des ensembles d'entraînement et de test
X_train.shape, X_test.shape

((782, 25), (196, 25))

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 782 entries, 784 to 102
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   NumberofBuildings                    782 non-null    float64
 1   NumberofFloors                       782 non-null    int64  
 2   PropertyGFATotal                     782 non-null    int64  
 3   LargestPropertyUseTypeGFA_pct        782 non-null    float64
 4   SecondLargestPropertyUseTypeGFA_pct  782 non-null    float64
 5   ThirdLargestPropertyUseTypeGFA_pct   782 non-null    float64
 6   BuildingAge                          782 non-null    int64  
 7   SteamUse_pct                         782 non-null    float64
 8   Electricity_pct                      782 non-null    float64
 9   NaturalGas_pct                       782 non-null    float64
 10  GroupedNeighborhood_A                782 non-null    bool   
 11  GroupedNeighborhood_B              

In [10]:
X_train.head()

,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseTypeGFA_pct,SecondLargestPropertyUseTypeGFA_pct,ThirdLargestPropertyUseTypeGFA_pct,BuildingAge,SteamUse_pct,Electricity_pct,NaturalGas_pct,...,GroupedLargType_C,GroupedLargType_D,GroupedSecondLargType_A,GroupedSecondLargType_B,GroupedSecondLargType_C,GroupedSecondLargType_D,GroupedThirdLargType_A,GroupedThirdLargType_B,GroupedThirdLargType_C,GroupedThirdLargType_D
784,1.0,1,22509,100.0,0.0,0.0,60,0.000000,100.000114,0.000000,...,False,True,False,False,False,True,False,False,False,True
909,1.0,5,36000,100.0,0.0,0.0,109,0.000000,27.255446,60.189450,...,True,False,False,False,False,True,False,False,False,True
33,1.0,1,389000,100.0,0.0,0.0,54,25.669501,60.745638,2.901303,...,True,False,False,False,False,True,False,False,False,True
31,1.0,2,93397,100.0,0.0,0.0,46,0.000000,33.488092,63.226530,...,True,False,False,False,False,True,False,False,False,True
731,1.0,2,25821,100.0,0.0,0.0,91,0.000000,97.556695,0.000000,...,False,True,False,False,False,True,False,False,False,True


In [11]:
y_train.head()

784    12.088487
909    14.055197
33     16.982542
31     16.646354
731    12.278165
Name: Log_SiteEnergyUseWN, dtype: float64

### Standardisation

In [12]:
# Initialisation du StandardScaler
scaler = StandardScaler()

In [13]:
columns_to_scale=['NumberofBuildings','NumberofFloors',	'PropertyGFATotal', 'LargestPropertyUseTypeGFA_pct', 
                  'SecondLargestPropertyUseTypeGFA_pct', 'ThirdLargestPropertyUseTypeGFA_pct',
                  'BuildingAge', 'SteamUse_pct', 'Electricity_pct', 'NaturalGas_pct']

In [14]:
# Séparer les colonnes à normaliser
X_train_to_scale = X_train[columns_to_scale]
X_test_to_scale = X_test[columns_to_scale]

# Appliquer la normalisation sur ces colonnes
X_train_scaled = scaler.fit_transform(X_train_to_scale)
X_test_scaled = scaler.transform(X_test_to_scale)

# Merge
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=columns_to_scale, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=columns_to_scale, index=X_test.index)
X_train_final = X_train.drop(columns=columns_to_scale).join(X_train_scaled_df)
X_test_final = X_test.drop(columns=columns_to_scale).join(X_test_scaled_df)


In [15]:
X_train_final.head()

,GroupedNeighborhood_A,GroupedNeighborhood_B,GroupedNeighborhood_C,GroupedLargType_A,GroupedLargType_B,GroupedLargType_C,GroupedLargType_D,GroupedSecondLargType_A,GroupedSecondLargType_B,GroupedSecondLargType_C,...,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseTypeGFA_pct,SecondLargestPropertyUseTypeGFA_pct,ThirdLargestPropertyUseTypeGFA_pct,BuildingAge,SteamUse_pct,Electricity_pct,NaturalGas_pct
784,False,False,True,False,False,False,True,False,False,False,...,-0.121177,-0.422615,-0.405920,0.604302,-0.586689,-0.313091,-0.013000,-0.233456,1.308473,-1.162566
909,False,False,True,False,False,True,False,False,False,False,...,-0.121177,0.266973,-0.313684,0.604302,-0.586689,-0.313091,1.573448,-0.233456,-1.336588,1.292339
33,True,False,False,False,False,True,False,False,False,False,...,-0.121177,-0.422615,2.099734,0.604302,-0.586689,-0.313091,-0.207259,2.726111,-0.118855,-1.044232
31,True,False,False,False,False,True,False,False,False,False,...,-0.121177,-0.250218,0.078733,0.604302,-0.586689,-0.313091,-0.466271,-0.233456,-1.109963,1.416210
731,False,False,True,False,False,False,True,False,False,False,...,-0.121177,-0.250218,-0.383276,0.604302,-0.586689,-0.313091,0.990671,-0.233456,1.219628,-1.162566


## Préparation des métriques

In [16]:
# Fonction pour calculer les métriques
def calc_metrics(y_true, y_pred):
    # Calcul du RMSE 
    rmse_log = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # Calcul du R-squared 
    r2_log = r2_score(y_true, y_pred)
    
    # Calcul du MAE 
    mae_log = mean_absolute_error(y_true, y_pred)
    
    return rmse_log, r2_log, mae_log

# Fonction pour afficher les métriques
def display_metrics(rmse_log, r2_log, mae_log):
    # Affichage des métriques
    print(f"RMSE : {rmse_log}")
    print(f"R-squared : {r2_log}")
    print(f"MAE : {mae_log}")

## Tests de modèles

### Régression linéaire (Baseline)

In [17]:
# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train_final, y_train)

# Prédiction sur le jeu de test
y_pred = model.predict(X_test_final)

print("Métriques pour le modèle moyen :")
metrics = calc_metrics(y_test, y_pred)
display_metrics(*metrics) 


Métriques pour le modèle moyen :
RMSE : 0.786732364825023
R-squared : 0.49684498726399995
MAE : 0.6211879353606653


### SVR

In [18]:
# Démarre le chronomètre
start_time = time.time()

# Création du modèle SVM pour la régression
svr_model = SVR()

# Entraînement du modèle sur les données d'entraînement
svr_model.fit(X_train_final, y_train)

# Prédiction sur le jeu de test
y_pred_svr = svr_model.predict(X_test_final)

# Arrête le chronomètre
end_time = time.time()

# Calcule la durée totale
duration = end_time - start_time

print(f"Le temps de calcul est de {duration:.2f} secondes")
print("Métriques pour Support Vector Regression:")
metrics = calc_metrics(y_test, y_pred_svr)
display_metrics(*metrics)

Le temps de calcul est de 0.03 secondes
Métriques pour Support Vector Regression:
RMSE : 0.7282064486518425
R-squared : 0.5689210548609113
MAE : 0.5628793375911912


### Forêt aléatoire

In [19]:
# Démarre le chronomètre
start_time = time.time()

# Création du modèle Random Forest
rf_model = RandomForestRegressor()

# Entraînement du modèle sur les données d'entraînement
rf_model.fit(X_train_final, y_train)

# Prédiction sur le jeu de test
y_pred_rf = rf_model.predict(X_test_final)

# Arrête le chronomètre
end_time = time.time()

# Calcule la durée totale
duration = end_time - start_time

print(f"Le temps de calcul est de {duration:.2f} secondes")
print("Métriques pour Random Forest:")
metrics = calc_metrics(y_test, y_pred_rf)
display_metrics(*metrics) 

Le temps de calcul est de 0.56 secondes
Métriques pour Random Forest:
RMSE : 0.7067706833928802
R-squared : 0.5939263360741589
MAE : 0.5564660635450059


In [20]:
importances_rf = rf_model.feature_importances_

# Création d'un DataFrame pour afficher l'importance des variables
features_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances_rf})
features_rf = features_rf.sort_values(by='Importance', ascending=False)

print(features_rf)

                                Feature  Importance
17              GroupedSecondLargType_A    0.604958
6                           BuildingAge    0.097829
23               GroupedThirdLargType_C    0.058869
21               GroupedThirdLargType_A    0.055267
24               GroupedThirdLargType_D    0.053055
16                    GroupedLargType_D    0.026200
3         LargestPropertyUseTypeGFA_pct    0.024950
19              GroupedSecondLargType_C    0.015248
18              GroupedSecondLargType_B    0.013821
5    ThirdLargestPropertyUseTypeGFA_pct    0.009930
2                      PropertyGFATotal    0.007350
0                     NumberofBuildings    0.006827
20              GroupedSecondLargType_D    0.005791
1                        NumberofFloors    0.003534
10                GroupedNeighborhood_A    0.003485
15                    GroupedLargType_C    0.003159
14                    GroupedLargType_B    0.002259
22               GroupedThirdLargType_B    0.001473
4   SecondLa

### Gradient Boosting

In [21]:
# Démarre le chronomètre
start_time = time.time()

# Création du modèle Gradient Boosting
gb_model = GradientBoostingRegressor()

# Entraînement du modèle sur les données d'entraînement
gb_model.fit(X_train_final, y_train)

# Prédiction sur le jeu de test
y_pred_gb = gb_model.predict(X_test_final)

# Arrête le chronomètre
end_time = time.time()

# Calcule la durée totale
duration = end_time - start_time

print(f"Le temps de calcul est de {duration:.2f} secondes")
print("Métriques pour Gradient Boosting:")
metrics = calc_metrics(y_test, y_pred_gb)
display_metrics(*metrics) 

Le temps de calcul est de 0.20 secondes
Métriques pour Gradient Boosting:
RMSE : 0.6796709264688118
R-squared : 0.6244695511094536
MAE : 0.5365228574229093


In [22]:
importances_gb = gb_model.feature_importances_

# Création d'un DataFrame pour afficher l'importance des variables
features_gb = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances_gb})
features_gb = features_gb.sort_values(by='Importance', ascending=False)

print(features_gb)

                                Feature  Importance
17              GroupedSecondLargType_A    0.665148
6                           BuildingAge    0.112422
24               GroupedThirdLargType_D    0.058798
3         LargestPropertyUseTypeGFA_pct    0.045319
23               GroupedThirdLargType_C    0.034873
21               GroupedThirdLargType_A    0.014908
18              GroupedSecondLargType_B    0.013597
16                    GroupedLargType_D    0.010864
10                GroupedNeighborhood_A    0.007283
19              GroupedSecondLargType_C    0.006632
5    ThirdLargestPropertyUseTypeGFA_pct    0.006288
2                      PropertyGFATotal    0.005128
20              GroupedSecondLargType_D    0.004247
14                    GroupedLargType_B    0.004049
15                    GroupedLargType_C    0.002739
4   SecondLargestPropertyUseTypeGFA_pct    0.002109
22               GroupedThirdLargType_B    0.001911
8                       Electricity_pct    0.000831
7           

<b>Modèle moyen</b> : <br>
Le modèle moyen sert de point de référence minimal pour évaluer les performances des autres modèles.

<b>Forêt Aléatoire</b> :<br>
Le modèle de forêt aléatoire a surpassé de manière significative la régression linéaire, avec un RMSE plus faible et un R-squared positif.
    
<b>Gradient Boosting</b> :<br>
Le modèle Gradient Boosting offre une amélioration légère mais significative sur la forêt aléatoire en termes de RMSE et de R-squared. 
Le Gradient Boosting est également plus rapide à calculer.

<b>Résumé</b> :
   - Le modèle moyen sert d'étalon de base, mais ses performances sont clairement surpassées par des modèles plus complexes.
   - La forêt aléatoire et le Gradient Boosting émergent comme les meilleurs candidats pour résoudre ce problème, avec un  avantage pour le gradient boosting.
 
    
Compte tenu de ces résultats, nous allons procéder à une validation croisée des modèles de Gradient Boosting et Random Forest pour confirmer le meilleur modèle.

## Validation croisée

In [23]:
# Métriques à utiliser
scoring = ['neg_mean_squared_error', 'r2', 'neg_mean_absolute_error']

# Pour Random Forest
rf_scores = cross_validate(rf_model, X_train_final, y_train, cv=5, scoring=scoring)
rf_rmse = np.sqrt(-rf_scores['test_neg_mean_squared_error'].mean())
rf_r2 = rf_scores['test_r2'].mean()
rf_mae = -rf_scores['test_neg_mean_absolute_error'].mean()

# Pour Gradient Boosting
gb_scores = cross_validate(gb_model, X_train_final, y_train, cv=5, scoring=scoring)
gb_rmse = np.sqrt(-gb_scores['test_neg_mean_squared_error'].mean())
gb_r2 = gb_scores['test_r2'].mean()
gb_mae = -gb_scores['test_neg_mean_absolute_error'].mean()

# Création d'un DataFrame pour afficher les métriques de Random Forest et Gradient Boosting
df_metrics = pd.DataFrame({
    'Metric': ['RMSE', 'R2', 'MAE'],
    'Random Forest': [rf_rmse, rf_r2, rf_mae],
    'Gradient Boosting': [gb_rmse, gb_r2, gb_mae]
})

# Afficher le DataFrame
df_metrics


,Metric,Random Forest,Gradient Boosting
0,RMSE,0.720686,0.696767
1,R2,0.662060,0.683900
2,MAE,0.540406,0.527964


La différence entre les métriques des deux modèles sont suffisamment significative pour confirmer le choix du Gradient Boosting.

## Recherche des hyperparamètres

In [24]:
# Définition des hyperparamètres à tester
param_dist = {
    'n_estimators': np.arange(50, 200, 10),
    'learning_rate': np.linspace(0.008, 0.2, 20),
    'max_depth': np.arange(2, 10, 1),
    'min_samples_split': np.arange(2, 10, 1),
    'min_samples_leaf': np.arange(1, 5, 1),
    'subsample': np.linspace(0.5, 0.9, 20)
}

# Initialisation de la recherche aléatoire
random_search = RandomizedSearchCV(GradientBoostingRegressor(), param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Exécution de la recherche aléatoire sur le jeu d'entraînement
random_search.fit(X_train_final, y_train)

# Récupération du meilleur modèle
best_gb_model = random_search.best_estimator_

# Affichage des meilleurs hyperparamètres
print(random_search.best_params_)

{'subsample': 0.9, 'n_estimators': 140, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_depth': 2, 'learning_rate': 0.06863157894736843}


In [25]:
# Définition de la grille d'hyperparamètres
param_grid = {
    'n_estimators': [100, 200, 250],
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [1, 2, 3],
    'min_samples_split': [6, 7, 8],
    'min_samples_leaf': [2, 3, 4],
    'subsample': [0.5, 0.6, 0.7]
}

# Configuration de GridSearchCV
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Exécution de la recherche d'hyperparamètres
grid_search.fit(X_train_final, y_train)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres :", grid_search.best_params_)

# Prédiction avec le meilleur modèle
y_pred = grid_search.best_estimator_.predict(X_test_final)

# Calcul des métriques pour le meilleur modèle
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("RMSE :", rmse)
print("R² :", r2)
print("MAE :", mae)

# Affichage des 3 meilleures configurations d'hyperparamètres
results = grid_search.cv_results_
for i in range(3):
    print(f"\nConfiguration {i+1}:")
    print("Paramètres :", results['params'][results['rank_test_score'][i]])
    print("RMSE :", np.sqrt(-results['mean_test_score'][results['rank_test_score'][i]]))

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Meilleurs paramètres : {'learning_rate': 0.05, 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 250, 'subsample': 0.6}
RMSE : 0.6715890361401696
R² : 0.6333472340837312
MAE : 0.5273345202233247

Configuration 1:
Paramètres : {'learning_rate': 0.15, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 6, 'n_estimators': 100, 'subsample': 0.5}
RMSE : 0.7062153172634754

Configuration 2:
Paramètres : {'learning_rate': 0.15, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 7, 'n_estimators': 250, 'subsample': 0.5}
RMSE : 0.7441202877492727

Configuration 3:
Paramètres : {'learning_rate': 0.15, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 250, 'subsample': 0.5}
RMSE : 0.7403357383966175


In [26]:
from sklearn.model_selection import cross_val_score

# Meilleurs hyperparamètres obtenus de GridSearchCV
best_params = grid_search.best_params_

# Configuration du modèle avec les meilleurs hyperparamètres
best_gb_model = GradientBoostingRegressor(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    subsample=best_params['subsample'],
    random_state=0
)

# Métriques à évaluer lors de la validation croisée
scoring_metrics = ['neg_mean_squared_error', 'r2', 'neg_mean_absolute_error']

# Exécution de la validation croisée
cv_results = cross_validate(best_gb_model, X_train_final, y_train, cv=5, scoring=scoring_metrics, return_train_score=True)

# Calcul et affichage des métriques moyennes pour chaque pli
rmse_scores = np.sqrt(-cv_results['test_neg_mean_squared_error'])
r2_scores = cv_results['test_r2']
mae_scores = -cv_results['test_neg_mean_absolute_error']

print(f"Validation Croisée RMSE: {rmse_scores.mean():.4f} (± {rmse_scores.std():.4f})")
print(f"Validation Croisée R²: {r2_scores.mean():.4f} (± {r2_scores.std():.4f})")
print(f"Validation Croisée MAE: {mae_scores.mean():.4f} (± {mae_scores.std():.4f})")


Validation Croisée RMSE: 0.6871 (± 0.0333)
Validation Croisée R²: 0.6915 (± 0.0287)
Validation Croisée MAE: 0.5215 (± 0.0314)


In [27]:
# Prédictions sur l'ensemble d'entraînement
y_train_pred = gb_model.predict(X_train_final)

# Prédictions sur l'ensemble de test
y_test_pred = gb_model.predict(X_test_final)

# Calcul des métriques pour l'ensemble d'entraînement
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)

# Calcul des métriques pour l'ensemble de test
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Affichage des métriques
print("Ensemble d'entraînement : RMSE = {:.4f}, R² = {:.4f}, MAE = {:.4f}".format(train_rmse, train_r2, train_mae))
print("Ensemble de test : RMSE = {:.4f}, R² = {:.4f}, MAE = {:.4f}".format(test_rmse, test_r2, test_mae))


Ensemble d'entraînement : RMSE = 0.5119, R² = 0.8312, MAE = 0.3858
Ensemble de test : RMSE = 0.6797, R² = 0.6245, MAE = 0.5365
